- open main folder
- convert all scans to nii
- read xlsx
- select corresponding scan Number

- coregistration afni
- rel. cbv with 3dcalc

In [3]:
import os
import subprocess
import numpy as np
import pandas as pd
import MRIhelperfunctions as h
import MRIhelperfunctions
from MRIhelperfunctions import Settings
from imp import importlib
import fnmatch

import papermill as pm

In [15]:
importlib.reload(h)
import MRIhelperfunctions as h
from MRIhelperfunctions import Settings

In [25]:
# these settings never change across modalities

raw_folder = '/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata'
main_folder = os.path.dirname(raw_folder) 

excel_filename = 'Animal_Scan_IDs.xlsx'

analysis_folder = os.path.join(main_folder,'Analysis')

notebookfolder = '/BigHDD/MR_DATA/GithubRepositories/CalciumfMRI/fMRI_notebooks'

folders = {}
folders.update({'main':main_folder, 'raw':raw_folder, 'analysis':analysis_folder, 'notebooks':notebookfolder, 'excel':excel_filename})
# folders = Settings(main = main_folder, raw = raw_folder, analysis = analysis_folder, notebooks = notebookfolder, excel = excel_filename)

In [16]:
info = h.getinfo(folders)


info

# for folderName, subfolders, filenames in os.walk(main_folder):
#     for file in filenames:
#         if (fnmatch.fnmatch(file, '*.nii') and 'analyzed' not in folderName and 'groupstats' not in folderName):
#             ID = h.splitall(folderName)[-1]
#             print(ID)
#             file_path1 = glob.glob(os.path.join(folderName,file))
#             file_path2 = os.path.dirname(file_path1[0]) # folder
#             file_path1 = re.sub(r'\+orig.BRIK.*.*',r'',file_path1[0])  # path to brik file without +orig ending
#             nb_kwargs = {'file_path': file_path1, 'folderName': file_path2, 'group': True}
#             print('running analysis for: ', file_path1)

            
            
# if not os.path.exists(groupstatsfolder):
#     os.makedirs(groupstatsfolder)

,glut1_male1,glut1_male2,glut1_f10wks1,glut1_f10wks2,glut1_f10wks3,glut1_f10wks4,glut1_f10wks5,glut1_f10wks6,glut1_f8wks1,glut1_f8wks2,glut1_f8wks3,glut1_f8wks4,glut1_f8wks5,glut1_f8wks6,glut1_f8wks7,glut1_f6wks2,glut1_f6wks1
scan,,,,,,,,,,,,,,,,,
Gd DCE,6,8,6,5,5,8,5,5,7,5,6,6,8,8,6,5,5
Angio pre,8,9,7,6,6,9,6,6,8,6,9,8,9,9,8,6,6
Angio post,12,12,11,9,9,13,10,11,12,11,12,12,16,13,12,9,10
Endorem DCE,11,11,10,8,8,12,9,10,10,10,11,11,15,12,11,8,9
Diamox,15,14,13,11,13,15,12,13,14,13,16,17,20,16,14,11,13
folder,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018060...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018060...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018060...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018060...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018060...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018060...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018060...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018060...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018061...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018061...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018061...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018061...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018061...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018061...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018061...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018061...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018061...


In [29]:
for animal in info.columns:   
    print('animal: ',animal)
    folders['animal']=info.loc['folder',animal]

    print('running Angiography analysis...')
    pm.execute_notebook(
       os.path.join(folders['notebooks'],'Angiography.ipynb'),
       os.path.join(folders['animal'],'Angiography_output.ipynb'),
       parameters = dict(folders=folders, animal=animal, info=info.to_json())
    )


   


animal:  glut1_male2
running Angiography analysis...


Input Notebook:  /BigHDD/MR_DATA/GithubRepositories/CalciumfMRI/fMRI_notebooks/Angiography.ipynb
Output Notebook: /home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/20180604_135437_Glut1_2_3/Angiography_output.ipynb






  0%|          | 0/24 [00:00<?, ?it/s]





 12%|█▎        | 3/24 [00:02<00:14,  1.41it/s]





 25%|██▌       | 6/24 [00:02<00:07,  2.50it/s]





 38%|███▊      | 9/24 [00:02<00:04,  3.57it/s]





 54%|█████▍    | 13/24 [01:48<01:31,  8.34s/it]





 62%|██████▎   | 15/24 [01:48<01:05,  7.26s/it]





 75%|███████▌  | 18/24 [01:49<00:36,  6.06s/it]





 83%|████████▎ | 20/24 [01:49<00:21,  5.48s/it]





 88%|████████▊ | 21/24 [01:49<00:15,  5.23s/it]





100%|██████████| 24/24 [01:49<00:00,  4.58s/it]

In [34]:
# collecting data from notebooks

for animal in info.columns[1:2]:   
    print('animal: ',animal)
    folders['animal']=info.loc['folder',animal]

    angio = pm.read_notebook(os.path.join(folders['animal'],'Angiography_output.ipynb'))
    info.loc['Angio_post_minus_pre', animal] = angio.data['post_minus_pre'] #location of the output .nii

info


animal:  glut1_male2


,glut1_male1,glut1_male2,glut1_f10wks1,glut1_f10wks2,glut1_f10wks3,glut1_f10wks4,glut1_f10wks5,glut1_f10wks6,glut1_f8wks1,glut1_f8wks2,glut1_f8wks3,glut1_f8wks4,glut1_f8wks5,glut1_f8wks6,glut1_f8wks7,glut1_f6wks2,glut1_f6wks1
scan,,,,,,,,,,,,,,,,,
Gd DCE,6,8,6,5,5,8,5,5,7,5,6,6,8,8,6,5,5
Angio pre,8,9,7,6,6,9,6,6,8,6,9,8,9,9,8,6,6
Angio post,12,12,11,9,9,13,10,11,12,11,12,12,16,13,12,9,10
Endorem DCE,11,11,10,8,8,12,9,10,10,10,11,11,15,12,11,8,9
Diamox,15,14,13,11,13,15,12,13,14,13,16,17,20,16,14,11,13
folder,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018060...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018060...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018060...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018060...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018060...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018060...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018060...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018060...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018061...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018061...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018061...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018061...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018061...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018061...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018061...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018061...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018061...
Angio_post_minus_pre,NaN,/home/aic/BigHDD/MR_DATA/GLUT1/Analysis/glut1_...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
